In [3]:
import csv
import datetime
import bcrypt  # For secure password hashing
from collections import defaultdict


class Book:
    """
    The Book class represents books in the library system.
    """
    def __init__(self, title, author, isbn, status="Available", borrower="", genre="Unknown", reading_time=0):
        self.title = title
        self.author = author
        self.isbn = isbn
        self.status = status
        self.borrower = borrower
        self.genre = genre
        self.reading_time = reading_time

    def __str__(self):
        return f"Title: {self.title}, Author: {self.author}, ISBN: {self.isbn}, Status: {self.status}, Genre: {self.genre}, Reading Time: {self.reading_time} hours"


class LibrarySystem:
    """
    The LibrarySystem class is the main interface for library operations.
    """
    def __init__(self):
        self.books_by_isbn = {}  # ISBN as key
        self.books_by_title = defaultdict(dict)  # Title as key, dictionary for multiple books
        self.books_by_author = defaultdict(dict)  # Author as key, dictionary for multiple books
        self.books_by_genre = defaultdict(dict)  # Genre as key, dictionary for multiple books
        self.user_activity = defaultdict(set)  # Track unique borrowed books per user
        self.users = {}  # Stores registered users with their hashed passwords
        self.logged_in_user = None  # To track the current logged-in user

    def start(self):
        """
        Loads the library data from a CSV file, skipping the header if present.
        """
        try:
            with open("library.csv", "r", newline='', encoding="utf-8") as file:
                reader = csv.reader(file)
                next(reader, None)  # Skip header row
                for row in reader:
                    if len(row) == 7:
                        try:
                            book = Book(row[0], row[1], row[2], row[3], row[4], row[5], int(row[6]))
                            self.addBook(book)  # Use the addBook method to ensure it's added correctly
                        except ValueError as e:
                            print(f"Skipping record due to value error: {row}. Error: {e}")
                    else:
                        print(f"Skipping malformed record: {row}")
            print("\nCity Library System is now running.\n")
        except FileNotFoundError:
            print("The library database file is missing. Starting with an empty library.")
    
    def exit(self):
        """
        Saves the library data to a CSV file when the system exits.
        """
        print(f"\n{self.getCurrentTime()} - Closing the City Library System.\n")
        try:
            with open("library.csv", "w", newline='', encoding="utf-8") as file:
                writer = csv.writer(file)
                writer.writerow(["Title", "Author", "ISBN", "Status", "Borrower", "Genre", "Reading Time"])
                for book in self.books_by_isbn.values():
                    writer.writerow([book.title, book.author, book.isbn, book.status, book.borrower, book.genre, book.reading_time])
        except Exception as e:
            print(f"Failed to write to library.csv: {e}")

    def addBook(self, book):
        """
        Adds a new book to the library if it's not already present.
        """
        if book.isbn in self.books_by_isbn:
            print(f"Book with ISBN {book.isbn} already exists in the system.")
            return
        print(f"{self.getCurrentTime()} - Adding book '{book.title}'.")
        self.books_by_isbn[book.isbn] = book
        self.books_by_title[book.title.lower()][book.isbn] = book
        self.books_by_author[book.author.lower()][book.isbn] = book
        self.books_by_genre[book.genre.lower()][book.isbn] = book
        print(f"Book '{book.title}' added successfully.")

    def borrowBook(self, isbn):
        if self.logged_in_user is None:
            print("Please log in to borrow books.")
            return
        print(f"{self.getCurrentTime()} - Borrowing attempt for ISBN {isbn} by {self.logged_in_user}.")
        book = self.getBookByISBN(isbn)
        if book:
            if book.status == "Available":
                book.status = "Borrowed"
                book.borrower = self.logged_in_user
                self.user_activity[self.logged_in_user].add(isbn)  # Track borrowed books by ISBN
                print(f"Book '{book.title}' is successfully borrowed by {self.logged_in_user}.")
            else:
                print(f"Book '{book.title}' is already borrowed by {book.borrower}.")
        else:
            print(f"No book found with ISBN {isbn}.")

    def returnBook(self, isbn):
        if self.logged_in_user is None:
            print("Please log in to return books.")
            return
        print(f"{self.getCurrentTime()} - Return attempt for ISBN {isbn}.")
        book = self.getBookByISBN(isbn)
        if book:
            if book.status == "Borrowed":
                book.status = "Available"
                borrower = book.borrower
                book.borrower = ""
                print(f"Book '{book.title}' is successfully returned by {borrower}.")
            else:
                print(f"Book '{book.title}' is already available.")
        else:
            print(f"No book found with ISBN {isbn}.")

    def displayBooks(self, page_number=1, books_per_page=5, filter_by=None, keyword=None):
        print(f"{self.getCurrentTime()} - Displaying books.")
        filtered_books = self.searchBooks(filter_by, keyword)
        self.paginateBooks(filtered_books, page_number, books_per_page)

    def displayAvailableBooks(self, page_number=1, books_per_page=5):
        print(f"{self.getCurrentTime()} - Displaying available books.")
        available_books = [book for book in self.books_by_isbn.values() if book.status.lower() == "available"]
        self.paginateBooks(available_books, page_number, books_per_page)

    def paginateBooks(self, books, page_number, books_per_page):
        total_books = len(books)
        total_pages = (total_books + books_per_page - 1) // books_per_page  # Round up to get total pages

        if total_books == 0:
            print("No books available for pagination.")
            return

        if page_number < 1 or page_number > total_pages:
            print(f"Invalid page number. Please select a page between 1 and {total_pages}.")
            return

        start_index = (page_number - 1) * books_per_page
        end_index = min(start_index + books_per_page, total_books)

        print(f"\nDisplaying page {page_number} of {total_pages}:")
        for i in range(start_index, end_index):
            book = books[i]
            print(f"{i + 1}. {book}")

        print(f"\nPage {page_number} of {total_pages}")
        if page_number > 1:
            print(f"Previous Page: {page_number - 1}")
        if page_number < total_pages:
            print(f"Next Page: {page_number + 1}")

    def score(self):
        print(f"{self.getCurrentTime()} - Displaying top users based on borrowing activity.")
        if not self.user_activity:
            print("\nNo borrowing activity recorded.")
            return
        
        top_users = sorted(self.user_activity.items(), key=lambda x: len(x[1]), reverse=True)  # Sort by number of borrowed books
        
        print("\nTop Users Based on Borrowing Activity:")
        for user, borrowed_books in top_users:
            print(f"{user}: {len(borrowed_books)} borrows")

    def getBookByISBN(self, isbn):
        """
        Returns a book by its ISBN using the lookup table.
        """
        return self.books_by_isbn.get(isbn, None)

    def searchBooks(self, filter_by=None, keyword=None):
        """
        Filters books based on the search filter and keyword using optimized lookup tables.
        """
        if not filter_by or not keyword:
            return list(self.books_by_isbn.values())  # No filter or keyword, return all books

        keyword = keyword.lower()  # Case-insensitive search
        filtered_books = []

        if filter_by == "title":
            filtered_books = list(self.books_by_title.get(keyword, {}).values())
        elif filter_by == "author":
            filtered_books = list(self.books_by_author.get(keyword, {}).values())
        elif filter_by == "genre":
            filtered_books = list(self.books_by_genre.get(keyword, {}).values())
        elif filter_by == "status":
            filtered_books = [book for book in self.books_by_isbn.values() if book.status.lower() == keyword]
        
        return filtered_books

    def getCurrentTime(self):
        """
        Returns the current date and time in a readable format.
        """
        current_time = datetime.datetime.now()
        return current_time.strftime("%Y-%m-%d %H:%M:%S")  # Format the time as "YYYY-MM-DD HH:MM:SS"

    def register_user(self, username, password):
        """
        Registers a new user with a hashed password.
        """
        if username in self.users:
            print("Username already exists!")
            return False
        hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
        self.users[username] = hashed_password
        print(f"{username} registered successfully.")
        return True

    def login_user(self, username, password):
        """
        Logs in a user by validating their credentials.
        """
        if username in self.users and bcrypt.checkpw(password.encode('utf-8'), self.users[username]):
            self.logged_in_user = username
            print(f"{username} logged in successfully.")
            return True
        print("Invalid username or password.")
        return False

    def logout_user(self):
        """
        Logs out the current user.
        """
        if self.logged_in_user:
            print(f"{self.logged_in_user} logged out.")
            self.logged_in_user = None
        else:
            print("No user is logged in.")

    def show_menu(self):
        """
        Displays the main menu and handles user input.
        """
        while True:
            if self.logged_in_user:
                print(f"\nWelcome {self.logged_in_user}!")
            else:
                print("\nWelcome to City Library System!")

            print("1. Register User")
            print("2. Login User")
            print("3. Logout User")
            print("4. Borrow Books")
            print("5. Return Books")
            print("6. Add Books")
            print("7. Display Available Books")
            print("8. Search For Books")
            print("9. View Top Users")
            print("10. Exit System")

            choice = input("How can we help? Please enter the number: ")

            if choice == "1" and not self.logged_in_user:
                self.register_user_interface()
            elif choice == "2" and not self.logged_in_user:
                self.login_user_interface()
            elif choice == "3":
                self.logout_user()
            elif choice == "4":
                self.borrow_book_interface()
            elif choice == "5":
                self.return_book_interface()
            elif choice == "6":
                self.add_book_interface()
            elif choice == "7":
                self.display_available_books_interface()
            elif choice == "8":
                self.search_books_interface()
            elif choice == "9":
                self.score()
            elif choice == "10":
                self.exit()
                print("\nThank you for using City Library System. See you again!")
                break
            else:
                print("\nInvalid input. Please try again.")

    def register_user_interface(self):
        username = input("Enter username: ")
        password = input("Enter password: ")
        self.register_user(username, password)

    def login_user_interface(self):
        username = input("Enter username: ")
        password = input("Enter password: ")
        if self.login_user(username, password):
            print(f"Welcome {username}!")

    def borrow_book_interface(self):
        isbn = input("Please enter the ISBN of the book you would like to borrow: ")
        self.borrowBook(isbn)

    def return_book_interface(self):
        isbn = input("Please enter the ISBN of the book you would like to return: ")
        self.returnBook(isbn)

    def add_book_interface(self):
        title = input("Enter the title of the book: ")
        author = input("Enter the author of the book: ")
        isbn = input("Enter the ISBN of the book: ")
        genre = input("Enter the genre of the book: ")
        while True:
            try:
                reading_time = int(input("Enter the estimated reading time (in hours): "))
                break
            except ValueError:
                print("Please enter a valid number for reading time.")
        new_book = Book(title, author, isbn, "Available", "", genre, reading_time)
        self.addBook(new_book)

    def display_available_books_interface(self):
        page_number = int(input("Enter page number to display: "))
        self.displayAvailableBooks(page_number=page_number, books_per_page=5)

    def search_books_interface(self):
        filter_by = input("You would like to search by (title/author/status/genre): ").strip().lower()
        keyword = input("Enter the keyword: ").strip().lower()
        self.displayBooks(filter_by=filter_by, keyword=keyword)


def system():
    library = LibrarySystem()
    library.start()
    library.show_menu()

system()


2024-12-14 17:16:52 - Adding book '1984'.
Book '1984' added successfully.
2024-12-14 17:16:52 - Adding book 'To Kill a Mockingbird'.
Book 'To Kill a Mockingbird' added successfully.
2024-12-14 17:16:52 - Adding book 'Brave New World'.
Book 'Brave New World' added successfully.
2024-12-14 17:16:52 - Adding book 'The Great Gatsby'.
Book 'The Great Gatsby' added successfully.
2024-12-14 17:16:52 - Adding book 'Moby Dick'.
Book 'Moby Dick' added successfully.
2024-12-14 17:16:52 - Adding book 'Pride and Prejudice'.
Book 'Pride and Prejudice' added successfully.
2024-12-14 17:16:52 - Adding book 'The Catcher in the Rye'.
Book 'The Catcher in the Rye' added successfully.
2024-12-14 17:16:52 - Adding book 'The Hobbit'.
Book 'The Hobbit' added successfully.
2024-12-14 17:16:52 - Adding book 'Harry Potter and the Sorcerer's Stone'.
Book 'Harry Potter and the Sorcerer's Stone' added successfully.
2024-12-14 17:16:52 - Adding book 'War and Peace'.
Book 'War and Peace' added successfully.
2024-12-

How can we help? Please enter the number:  1
Enter username:  Cindy
Enter password:  123456


Cindy registered successfully.

Welcome to City Library System!
1. Register User
2. Login User
3. Logout User
4. Borrow Books
5. Return Books
6. Add Books
7. Display Available Books
8. Search For Books
9. View Top Users
10. Exit System


How can we help? Please enter the number:  2
Enter username:  Cindy
Enter password:  123456


Cindy logged in successfully.
Welcome Cindy!

Welcome Cindy!
1. Register User
2. Login User
3. Logout User
4. Borrow Books
5. Return Books
6. Add Books
7. Display Available Books
8. Search For Books
9. View Top Users
10. Exit System


How can we help? Please enter the number:  4
Please enter the ISBN of the book you would like to borrow:  2


2024-12-14 17:17:28 - Borrowing attempt for ISBN 2 by Cindy.
Book '2' is successfully borrowed by Cindy.

Welcome Cindy!
1. Register User
2. Login User
3. Logout User
4. Borrow Books
5. Return Books
6. Add Books
7. Display Available Books
8. Search For Books
9. View Top Users
10. Exit System


How can we help? Please enter the number:  10



2024-12-14 17:19:37 - Closing the City Library System.


Thank you for using City Library System. See you again!
